In [53]:
import glob
import regex as re
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [41]:
with open('../data/headers_mod.csv') as f:
    mod_headers = f.readlines()[0].split()
    
with open('../data/headers_obs.csv') as f:
    obs_headers = f.readlines()[0].split()
    
mod_headers, obs_headers

In [42]:
targets = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')

In [43]:
targets.head()

,date,station,target
0,2015-01-03,ES0691A,0
1,2015-01-03,ES1396A,0
2,2015-01-03,ES1438A,0
3,2015-01-03,ES1480A,0
4,2015-01-03,ES1679A,0


In [44]:
stations.head()

,code,name,lat,lon,height,city,country
0,ES1480A,Barcelona (Gràcia-Sant Gervasi),41.3987,2.15339,57,Barcelona,España
1,ES1856A,Barcelona (Vall d'Hebron),41.4261,2.14799,136,Barcelona,España
2,ES1679A,Ciutadella,41.3864,2.18742,7,Barcelona,España
3,ES1438A,Barcelona (Eixample),41.3853,2.15382,26,Barcelona,España
4,ES1396A,Barcelona,41.3788,2.13310,35,Barcelona,España


In [67]:
observations = []
for folder in glob.glob('../data/obs/*'):
    for file in glob.glob('{}/*'.format(folder)):
        year = re.search('\d{4}', file)[0]
        station = 'ES{}A'.format(re.findall('\d{4}', file)[1])
        data = pd.read_csv(file, sep='\t')
        observations.append((year, station, data))
        
observations[0][2].sample(5)

,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollutant/8,hour,12,µg/m3,2013-02-13 01:00:00,2013-02-13 02:00:00,1,1.1
2004,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,10,µg/m3,2013-08-03 14:00:00,2013-08-03 15:00:00,1,1
6948,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,14,µg/m3,2013-04-21 07:00:00,2013-04-21 08:00:00,1,1
2719,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,34,µg/m3,2013-09-02 14:00:00,2013-09-02 15:00:00,1,1
8041,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,27,µg/m3,2013-06-03 12:00:00,2013-06-03 13:00:00,1,1
7408,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1992A,ES1992A,SP_08019057_8_8,SPP_08019057_8_8.1,SAM_08019057_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,38,µg/m3,2013-05-07 23:00:00,2013-05-08 00:00:00,1,1
